In [ ]:
#!pip install -q transformers accelerate datasets

In [1]:
import torch
import gc
import traceback
from google.colab import files, drive
import shutil
from transformers import pipeline,WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# -- ENV --
# Check if GPU is available and set the device --> likely will need GPU with full run
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

cv_sample_path = "/content/drive/MyDrive/Capstone/Data/Common Voices Sample/common_voices_sample1.csv"
audio_path =  "/content/drive/MyDrive/Capstone/Data/Common Voices Clips"
output_path = "/content/drive/MyDrive/transcripts_twenties.csv"

Device: cpu


In [4]:
df = pd.read_csv(cv_sample_path)
df

,Unnamed: 0,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,0,108aa26d791cf89814686b002c23f140ee6de5b2e6c8a9...,common_voice_en_20131778.mp3,30a0dc4d58a30bb1db8f5c38095bd04c90d2fa598ae06f...,Lloyd's of London is a big participant in this...,NaN,2,0,twenties,male_masculine,NaN,NaN,en,NaN
1,1,b5c6eb9fcb6cf714c4cef913529d45f8ffe4214e6c4305...,common_voice_en_24569804.mp3,823261ec158ebb3ed0fb56f13b1ace9884e266458dd8e5...,She played the flute for over ten years.,NaN,2,0,twenties,male_masculine,NaN,NaN,en,NaN
2,2,15f4b2d89fd6ef677ad0cf16118393247db64e892060c2...,common_voice_en_27411002.mp3,ab3f945540310f2242082bc6b12bbf0f0e40b09c2e9a92...,"All songs written by Dave Chandler, except whe...",NaN,2,1,twenties,male_masculine,"India and South Asia (India, Pakistan, Sri Lanka)",NaN,en,NaN
3,3,1fd8f855cb79a3775218a82cb652aa3f6bb9a0b462b2c6...,common_voice_en_25918568.mp3,01ef759712452b4829af7bf16c4984f2b3142ec5ae9450...,You tourniquet dead tissue so that it doesn’t ...,NaN,3,2,twenties,male_masculine,NaN,NaN,en,NaN
4,4,c7fa9b2502b716b516b076ca89ff79e29d183ac0c006ce...,common_voice_en_19826146.mp3,2680dc9fed4f87d4530db799bc7dd01740b6ebc58b91c3...,Mossell practiced law with two African-America...,NaN,4,0,twenties,male_masculine,England English,NaN,en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,a832c76013f233e214baeefa9691b681581684f542fc8a...,common_voice_en_37971557.mp3,d2e98cd1ba67157d5f6c6d0731f28f9ae2fb8605a6a1ef...,"Arthur L. Keith, reviewing H. J.",NaN,4,0,nineties,male_masculine,England English,NaN,en,NaN
796,796,a832c76013f233e214baeefa9691b681581684f542fc8a...,common_voice_en_37971817.mp3,d2f2d0982f43d994ab59041b632afb571ab82c5b290f91...,"Working with the Navy, he helped develop high-...",NaN,2,0,nineties,male_masculine,England English,NaN,en,NaN
797,797,9ef63c9b9fc1b47bd05e7a351efaed8fb08729b6c80d8c...,common_voice_en_28069527.mp3,b7094fd77fcf8e1d08982a9cc8c6dd2471e1a5477da93b...,Aldridge put up similar numbers to the previou...,NaN,2,0,nineties,NaN,United States English,NaN,en,NaN
798,798,1614deccdbfff77e69c0d1b186e5dfc4de1c54b80c6b71...,common_voice_en_18714224.mp3,01f4670c16252fc0950c74b19966669ee34532167f5354...,"Many of the species are highly aromatic, and t...",NaN,2,1,nineties,male_masculine,United States English,NaN,en,NaN


In [5]:
def get_full_path(base_audio_path: str) -> str:
    """
    Takes 'path' in Common voice sample (e.g., 'common_voice_en_42696165.mp3') and
    joins it with audio_path and tries common extensions if missing.
    """
    # If 'path' already contains directories, don't double-join
    fname = os.path.basename(str(base_audio_path))
    full = os.path.join(audio_path, fname)

    # Try extensions if missing
    if not os.path.exists(full):
        base, ext = os.path.splitext(full)
        if ext == "":
            for cand_ext in [".mp3", ".wav", ".flac", ".m4a", ".ogg"]:
                cand = base + cand_ext
                if os.path.exists(cand):
                    return cand
    return full

In [6]:
#Add full paths column
df["full_audio_path"] = df["path"].apply(get_full_path)


missing = []
for p in df["full_audio_path"]:
    if os.path.exists(p):
        print("ok -->", p)
    else:
      missing.append(p)
        #print("missing -->", p)


print("\nSummary:")
print(f"Total rows: {len(df)}")
print(f"Found files: {len(df) - len(missing)}")
print(f"Missing files: {len(missing)}")

OK     → /content/drive/MyDrive/Capstone/Data/Common Voices Clips/common_voice_en_43196392.mp3

Summary:
Total rows: 800
Found files: 1
Missing files: 799


In [7]:
# Using Base but can use Large later
model = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-base",
  device=device,
  #chunk_length_s = 30, #duration of audio / experiment
  #stride_length_s = (5,5), #left and right overlap
  return_timestamps=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
found_df = df[df["full_audio_path"].apply(os.path.exists)].copy()
print(f"Found {len(found_df)} files (out of {len(df)} rows).")

Found 1 files (out of 800 rows).


In [9]:
results = []
for _, row in tqdm(found_df.iterrows(), total=len(found_df)):
    audio_path = row["full_audio_path"]
    out = model(audio_path)

    results.append({
        "path": row.get("path", os.path.basename(audio_path)),
        "full_audio_path": audio_path,
        "age": row.get("age", None),
        "gender": row.get("gender", None),
        "sentence": row.get("sentence", None),
        "transcribed_text": out.get("text", "")
    })

    # Print as you go --> change when we have more data
    print("\n=== ", os.path.basename(audio_path), " ===")
    print(results[-1]["transcribed_text"])

  0%|          | 0/1 [00:00<?, ?it/s]`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
100%|██████████| 1/1 [00:09<00:00,  9.49s/it]


===  common_voice_en_43196392.mp3  ===
 He played college football at Iowa.


In [10]:
results
out_df = pd.DataFrame(results)
out_df

,path,full_audio_path,age,gender,sentence,transcribed_text
0,common_voice_en_43196392.mp3,/content/drive/MyDrive/Capstone/Data/Common Vo...,eighties,NaN,He played college football at Iowa.,He played college football at Iowa.


In [11]:
out_df.to_csv("/content/drive/MyDrive/transcripts_found.csv", index=False)

In [27]:
# audio_file = "/content/drive/MyDrive/Capstone/Data/Common Voices Clips/common_voice_en_42696165.mp3"
# result = pipe(audio_file)
# print("\nTranscription:")
# print(result["text"])

`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.



Transcription:
 The division was then commanded by Major General John Hawksworth.
